In [2]:
#Dylan Coulter
#CS340
#Assignment: Module 7 Final Project
#Resource:https://dash.plotly.com/datatable/interactivity
#Resource: https://github.com/plotly/dash/issues/136
#Resource: https://stackoverflow.com/questions/6461541/multiple-divs-inside-a-container-div
#Resource: https://plotly.com/python/pie-charts/
#Image resource:https://vombanachk9.com/dogs-in-action/search-rescue/
#Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

# Configure OS routines
import os

# Configure the plotting routines
import dash
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter.animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Dc7654321"
HOST = 'nv-desktop-services.apporto.com'
PORT = 30968
DB = 'AAC'
COL = 'animals'
shelter = AnimalShelter(username, password)

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
# replace with your own image
logo_file = 'Grazioso_Salvare_Logo.png'
encoded_logo = base64.b64encode(open(logo_file, 'rb').read())
#Image resource:https://vombanachk9.com/dogs-in-action/search-rescue/
logo_file_two = 'rescue_animal.jpg'
encoded_logo_two = base64.b64encode(open(logo_file_two, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard', style={'color': 'teal'}))),
    html.Center(html.B(html.H2("Dylan Coulter SNHU CS340 MongoDB Authentication", style={'color': 'blue'}))),
    html.Center(html.Img(style={'height':'30%', 'width':'30%'}, src="data:image/png;base64,{}".format(encoded_logo.decode()))),
    html.Center(html.B(html.H2("Training animals that could one day save your life!", style={'color': 'Red'}))),
    html.Center(html.Img(style={'height':'50%', 'width':'50%'}, src="data:image/png;base64,{}".format(encoded_logo_two.decode()))),   
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

        html.Hr(),
        html.Div([
        html.H4(html.Center(html.Label(['Filter Options: '], style={'font-weight': 'italics'}))),
        html.H4(html.Center(dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue    ', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue    ', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking    ', 'value': 'Disaster'},
                {'label': 'Reset    ', 'value': 'Reset'}
            ],
            value=4,
            labelStyle={'display': 'block'}
        ))
    )]),
            
    html.Br(),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

 dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #Resource:https://dash.plotly.com/datatable/interactivity
        editable=True,#allows for cell editing
        filter_action="native", #filtering by column
        sort_action="native", #sorting by column
        sort_mode="multi", #more than one field can be sorted
        column_selectable=False,
        row_selectable="single",#single row selection
        row_deletable=False,#no deleting rows
        selected_columns=[],
        selected_rows=[0], #selected row so no call back errors
        page_action="native",#paging front-end
        page_current=0, #sets starting page
        page_size=10 # 10 rows per page
    ),
    html.Br(),
    html.Hr(),

#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    #Resource: https://github.com/plotly/dash/issues/136
    #Resource: https://stackoverflow.com/questions/6461541/multiple-divs-inside-a-container-div
    html.Center(html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                         
                         dcc.Graph(
                             id='graph-id',
                             className='col s12 m6',
                        style={
                        "width": "1000px",
                        "height": "800px",
                        "display": "inline-block",
                            
                            }
                         ),
                       
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                        style={
                        "display": "inline-block",
                        
                            }
                         ),
                     ]
                 ),
             ])
        )
])
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#Resource: https://dash.plotly.com/datatable/interactivity
        #Water rescue filter option
        #Adding in animals for the provided data table
        if filter_type == 'Water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                #"age_upon_outcome_in_weeks used to meet criteria stated in specifications document.
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Mountain or Wilderness Rescue filter option
        #Adding in animals for the provided data table
        elif filter_type == 'Mountain':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                #"age_upon_outcome_in_weeks used to meet criteria stated in specifications document.
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Disaster Rescue or Individual Tracking filter option
        #Adding in animals for the provided data table
        elif filter_type == 'Disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                #"age_upon_outcome_in_weeks used to meet criteria stated in specifications document.
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        #Reset filter option
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        #removing the last column
        df.drop(columns=['_id'],inplace=True)
        
        return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
#Resource: https://dash.plotly.com/datatable/interactivity

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

#Resource: https://plotly.com/python/pie-charts/
#creating the pie chart
@app.callback(
    Output("graph-id", "figure"),
    [Input("datatable-id", "data")]
)
#Generating the pie chart
def generate_chart(data):
    #code for pie-chart
    df = pd.DataFrame.from_dict(data)
    fig = px.pie(df,names= 'breed')
    return fig
           

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart   
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]


    #Matching the values from the provided csv file
    return [
            dl.Map(style={'width': '1000px', 'height': '800px'},
               center=[30.75,-97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               # Marker with tool tip and popup
               # Column 13 and 14 define the grid-coordinates for 
               # the map
               # Column 4 defines the breed for the animal
               # Column 9 defines the name of the animal
               dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                  children=[
                  dl.Tooltip(dff.iloc[row,4]),
                  dl.Popup([
                     html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                 ])
              ])
           ])
        ]




if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:15732/
